In [17]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import RobustScaler
from keras.models import load_model
from joblib import dump, load

In [18]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


# GPU Configuration
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    try:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(e)
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Force CPU usage


GPU memory growth enabled


# current working sequence 
## Assuming model is trained till time x //
## So assume X+15 //
## Upload it //
## wait 15 min //
## get new data //
## add it to the list // 
## scale it with load->save 
## finetune it
# repeat 

# GLOBAL VARIABLES

In [19]:
split = 0.85
sequence_length = 60
epochs = 50
learning_rate = 0.0001

In [20]:
def init():
    stock_data = pd.read_csv("runningData15.csv", delimiter=",")
    close_prices = stock_data['close'].values.reshape(-1, 1) 
    scaler = load('scaler.joblib')
    test_scaled = scaler.transform(close_prices)
    dump(scaler, 'scaler.joblib')

    X_test_new = []
    y_test_new = []
    #Dummy
    y_test_new.append(test_scaled[sequence_length -1])
    len_test = len(test_scaled)
    print(test_scaled.shape)
    for i in range(len_test - sequence_length + 1 ):
        X_test_new.append(test_scaled[i:i + sequence_length].flatten())  # Flatten the sequence
        if i + sequence_length < len_test:
            y_test_new.append(test_scaled[i + sequence_length])
    X_test_new = np.array(X_test_new)
    y_test_new = np.array(y_test_new)
    y_test_new_original = scaler.inverse_transform(y_test_new.reshape(-1, 1))
    
    return X_test_new, y_test_new, y_test_new_original

In [21]:
scaler = load('scaler.joblib')

x, y, y_original = init()
x_og = scaler.inverse_transform(x[-1].reshape(-1, 1))
print(x_og)
print("====")
print(y_original[-1])

(2083, 1)
[[3308.57 ]
 [3309.305]
 [3314.065]
 [3321.965]
 [3325.44 ]
 [3326.395]
 [3330.94 ]
 [3331.11 ]
 [3323.1  ]
 [3328.235]
 [3326.87 ]
 [3328.77 ]
 [3328.205]
 [3326.665]
 [3332.105]
 [3337.005]
 [3339.465]
 [3337.715]
 [3338.965]
 [3338.175]
 [3336.71 ]
 [3334.55 ]
 [3338.675]
 [3339.25 ]
 [3343.18 ]
 [3354.485]
 [3353.045]
 [3353.01 ]
 [3351.22 ]
 [3350.065]
 [3351.19 ]
 [3351.615]
 [3350.2  ]
 [3345.52 ]
 [3348.855]
 [3342.905]
 [3345.42 ]
 [3353.025]
 [3348.205]
 [3342.9  ]
 [3324.095]
 [3335.19 ]
 [3337.365]
 [3331.84 ]
 [3340.525]
 [3340.96 ]
 [3342.855]
 [3342.03 ]
 [3344.635]
 [3343.84 ]
 [3341.94 ]
 [3338.66 ]
 [3341.595]
 [3336.93 ]
 [3336.96 ]
 [3333.395]
 [3329.48 ]
 [3330.855]
 [3328.045]
 [3331.685]]
====
[3331.685]


# Step 1 Predicting and uploading 

In [22]:
import requests
import numpy as np
from datetime import datetime
from uuid import UUID
from typing import Optional
from pydantic import BaseModel

class PredictionRequest(BaseModel):
    stock_id: Optional[UUID] = None
    symbol: Optional[str] = None
    opening_price: Optional[float] = None
    closing_price: Optional[float] = None
    high_price: Optional[float] = None
    low_price: Optional[float] = None
    volume: Optional[float] = None
    date: Optional[datetime] = None
    prediction_direction: Optional[bool] = None

In [23]:
def direction(y_original, y_pred):
    if y_original < y_pred:
        return True
    else:
        return False

In [24]:
def upload_prediction(y_test_new_original, results):
    API_URL = "http://127.0.0.1:8000/api/v1/prediction"
    HEADERS = {
        "Content-Type": "application/json",
        "Authorization": "Bearer your_api_key_here"
    }
    
    # Safely extract the last result value
    last_result = results[-1]
    if isinstance(last_result, np.ndarray):
        last_result = float(last_result.item())  # Properly extract scalar from array
    
    # Create the prediction request with Pydantic v2 syntax
    prediction_request = PredictionRequest(
        stock_id='ade11ce0-a353-427d-9ae7-26b948454eab',
        symbol="XAUUSD",
        opening_price=0.0,
        closing_price=last_result,
        high_price=0.0,
        low_price=0.0,
        volume=0.0,
        date=datetime.now(),
        prediction_direction=direction(y_test_new_original[-1], last_result)
    )
    
    # Convert to dict with proper serialization (Pydantic v2 syntax)
    request_data = prediction_request.model_dump()
    
    # Ensure proper serialization of special types
    if request_data['stock_id']:
        request_data['stock_id'] = str(request_data['stock_id'])
    if request_data['date']:
        request_data['date'] = request_data['date'].isoformat()
    
    try:
        response = requests.post(
            API_URL,
            json=request_data,
            headers=HEADERS,
            timeout=10
        )
        
        if response.status_code == 200:
            print("Prediction uploaded successfully")
            return True
        else:
            print(f"Failed to upload prediction: {response.status_code} - {response.text}")
            return False
    except Exception as e:
        print(f"Error uploading prediction: {str(e)}")
        return False

In [25]:
# results is like result =[], python list

def assume_and_upload(model, X_test, y_test, scaler, y_test_original, results):
    current_input = X_test[-1].reshape(1, sequence_length)
    prediction = model.predict(current_input, verbose=0)
    results.append(scaler.inverse_transform(prediction.reshape(-1, 1)))

    print(f"Predicted: {results[-1]}")

    upload_prediction(y_test_original, results)
    return results



# get New Data


In [26]:
def getLiveData(symbol: str, exchange: str, interval: str, n_bars: int):


        from tvDatafeed import TvDatafeedLive, Interval
        import pandas as pd


        username = 'your_actual_username'
        password = 'your_actual_password'

        tvl = TvDatafeedLive()

        xauusd_data = tvl.get_hist(symbol, exchange, interval=Interval.in_15_minute, n_bars=n_bars, fut_contract=None, extended_session=False, timeout=-1)
        # print(xauusd_data)

        # seis = tvl.new_seis('ETHUSDT', 'BINANCE', Interval.in_1_hour)
        # seis2 = tvl.new_seis('ETHUSDT', 'BINANCE', Interval.in_2_hour)

        # ethusdt_data = seis.get_hist(n_bars=10, timeout=-1)
        df = pd.DataFrame(xauusd_data)

        # Save to CSV
        df.to_csv('lastData.csv', index=True)

        # print("Data saved to lastData.csv")
        return df


In [27]:
import time
def get_new_data():
    while True:
        try:
            # Get new data
            getLiveData('XAUUSD', 'FOREXCOM', '15MIN', 1)
            
            # Read data - ensure no index is read
            main_df = pd.read_csv('runningData15.csv')
            new_df = pd.read_csv('lastData.csv')
            
            # Convert datetime columns
            main_df['datetime'] = pd.to_datetime(main_df['datetime'])
            new_df['datetime'] = pd.to_datetime(new_df['datetime'])
            
            # Sort by datetime to ensure chronological order
            main_df = main_df.sort_values('datetime')
            new_df = new_df.sort_values('datetime')
            
            # Get timestamps
            last_datetime = main_df['datetime'].iloc[-1]
            new_datetime = new_df['datetime'].iloc[0]
            
            if new_datetime > last_datetime:
                # Verify and clean new data before appending
                new_df = new_df[new_df['datetime'] > last_datetime]
                
                # Combine data
                combined_df = pd.concat([main_df, new_df], ignore_index=True)
                
                # Save without index and with exact column order
                combined_df.to_csv(
                    'runningData15.csv',
                    index=False,
                    columns=['datetime', 'symbol', 'open', 'high', 'low', 'close', 'volume'],
                    date_format='%Y-%m-%d %H:%M:%S'
                )
                print(f"✅ Successfully appended {len(new_df)} new records")
                print(f"Last timestamp now: {new_df['datetime'].iloc[-1]}")
                break
                
            elif new_datetime == last_datetime:
                # print("🔄 Data already exists. No update needed.")
                time.sleep(60)
                continue
                
            else:
                # print(f"⏳ New data ({new_datetime}) is older than last record ({last_datetime})")
                time.sleep(60)
                continue
                
        except pd.errors.EmptyDataError:
            print("⚠️ No data found in files. Retrying...")
            time.sleep(60)
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            time.sleep(60)

In [28]:
get_new_data()

KeyboardInterrupt: 

# FIINTUNE

In [29]:
def finetune(model, X_test, y_test, scaler, y_test_original, results):
    second_last = X_test[-2].reshape(1, sequence_length)
    actual = y_test[-1].reshape(1, 1)
    start_time = time.time()
    model.fit(second_last,
            actual,
            epochs=20,
            batch_size = 4048,
            verbose=0)
    model.save('fine_tune_model_march_27_2nd_trial.h5')
    end_time = time.time()
    print(f"Model finetuned in {end_time - start_time} seconds")
    return results


In [ ]:
# model = load_model('fine_tune_model_march_27_2nd_trial.h5')
# model.compile(
#     loss=tf.keras.losses.MeanSquaredError(),
#     optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate * 0.1)  # Lower learning rate for fine-tuning
# )
# x_test, y_test, y_test_original = init()
# results = []
# results = finetune(model, x_test, y_test, scaler, y_test_original, results)

(831, 1)


NameError: name 'scaler' is not defined

# Server

In [31]:
model = load_model('fine_tune_model_march_27_2nd_trial.h5')
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate * 0.1)  # Lower learning rate for fine-tuning
)
while(True):
    
    X_test_new, y_test_new, y_test_new_original = init()
    results = []
    results = assume_and_upload(model, X_test_new, y_test_new, scaler, y_test_new_original, results)
    # assume_and_upload(model, X_test_new, y_test_new, scaler, y_test_new_original, results)
    results = finetune(model, X_test_new, y_test_new, scaler, y_test_new_original, results)
    current_timestamp = time.time()
    current_time = time.localtime(current_timestamp)
    current_minute = current_time.tm_min  # Extract minute (0-59)
    remainder = 15 - (current_minute % 15)
    time.sleep((remainder - 1) * 60)
    get_new_data()
    

(2087, 1)
Predicted: [[3323.7952]]
Failed to upload prediction: 201 - {"message":"Prediction created successfully"}


Model finetuned in 2.707796096801758 seconds


✅ Successfully appended 1 new records
Last timestamp now: 2025-04-17 13:45:00
(2088, 1)
Predicted: [[3323.6777]]
Failed to upload prediction: 201 - {"message":"Prediction created successfully"}


Model finetuned in 0.6252148151397705 seconds


✅ Successfully appended 1 new records
Last timestamp now: 2025-04-17 14:00:00
(2089, 1)
Predicted: [[3326.1433]]
Failed to upload prediction: 201 - {"message":"Prediction created successfully"}


...

✅ Successfully appended 1 new records
Last timestamp now: 2025-04-18 02:30:00
(2139, 1)
Predicted: [[3330.6738]]
Failed to upload prediction: 201 - {"message":"Prediction created successfully"}


Model finetuned in 0.6544244289398193 seconds


✅ Successfully appended 1 new records
Last timestamp now: 2025-04-18 02:45:00
(2140, 1)
Predicted: [[3327.7178]]
Failed to upload prediction: 201 - {"message":"Prediction created successfully"}


Model finetuned in 0.6452250480651855 seconds


KeyboardInterrupt: 

In [ ]:
# model = load_model('fine_tune_model_march_27.h5')
# model.compile(
#         loss=tf.keras.losses.MeanSquaredError(),
#         optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate * 0.1)  # Lower learning rate for fine-tuning
#     )



In [ ]:
# results = assume_and_upload(model, x, y, scaler, y_original, [])
# print(results)

Predicted: [[3024.6055]]
Failed to upload prediction: 201 - {"message":"Prediction created successfully"}
[array([[3024.6055]], dtype=float32)]
